In [34]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Lambda, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K



In [3]:
# Define file path
csv_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/combined_file.csv"

# Load CSV file
df = pd.read_csv(csv_path)

# Display column names and first few rows
print("Dataset Columns:\n", df.columns)
print("\nFirst 5 Rows:\n", df.head())

Dataset Columns:
 Index(['timestamp', 'rcs', 'distance', 'angleAzimuth', 'angleElevation',
       'x_det', 'y_det', 'z', 'radialVelocity', 'objectID',
       'radialVelocityDomainMax', 'orientation', 'x_obj', 'y_obj',
       'width_edge_mean', 'length_edge_mean', 'object_id', 'header.origin.x',
       'header.origin.y', 'header.origin.z', 'header.origin.roll',
       'header.origin.pitch', 'header.origin.yaw', 'reference_point',
       'yaw_rate', 'ego_speed', 'centroid_x', 'centroid_y',
       'is_valid_detection'],
      dtype='object')

First 5 Rows:
       timestamp                                                rcs  \
0  2.746754e+07  [-18 -32 -25 -21 -32 -29 -36 -37 -31 -19  -5 -...   
1  2.746755e+07  [-23 -23 -22  -7 -28 -39 -25 -26 -23 -26 -27 -...   
2  2.746755e+07  [-35 -28 -25 -26 -35 -40 -28 -24 -32 -32 -25 -...   
3  2.746755e+07  [-34 -23 -24 -27 -33 -34 -24 -26 -31 -24 -38 -...   
4  2.746755e+07  [-28 -28 -26 -28 -26 -26 -30 -26 -31 -25 -26 -...   

                  

In [8]:


# Convert string representations of lists to actual lists if needed
def convert_to_list(value):
    """Ensure values are lists. If string, convert to list of floats."""
    if isinstance(value, list):  # Already a list, return as-is
        return value
    elif isinstance(value, str):  # Convert string to list
        return [float(x) for x in value.replace("[", "").replace("]", "").split()]
    else:
        return []  # Return empty list if unexpected format

# Apply conversion to detection columns
for col in ["x_det", "y_det", "radialVelocity", "rcs"]:
    df[col] = df[col].apply(convert_to_list)

# Convert label columns (single value per row)
def extract_single_value(value):
    """Extract a single float from a list or string representation"""
    if isinstance(value, list):  
        return float(value[0])  # Extract first value if already list
    elif isinstance(value, str):  
        return float(value.replace("[", "").replace("]", ""))  # Remove brackets and convert
    else:
        return float(value)  # Convert directly if already a float

# Apply conversion to label columns
for col in ["orientation", "width_edge_mean", "length_edge_mean", "centroid_x", "centroid_y"]:
    df[col] = df[col].apply(extract_single_value)

# Ensure scalar feature is float
df["ego_speed"] = df["ego_speed"].astype(float)

# Find maximum detections across all rows
max_detections = max(df["x_det"].apply(len).max(),
                     df["y_det"].apply(len).max(),
                     df["radialVelocity"].apply(len).max(),
                     df["rcs"].apply(len).max())

print(f"Maximum number of detections per row: {max_detections}")
print("\nSample Processed Data:\n", df.head())

Maximum number of detections per row: 995

Sample Processed Data:
       timestamp                                                rcs  \
0  2.746754e+07  [-18.0, -32.0, -25.0, -21.0, -32.0, -29.0, -36...   
1  2.746755e+07  [-23.0, -23.0, -22.0, -7.0, -28.0, -39.0, -25....   
2  2.746755e+07  [-35.0, -28.0, -25.0, -26.0, -35.0, -40.0, -28...   
3  2.746755e+07  [-34.0, -23.0, -24.0, -27.0, -33.0, -34.0, -24...   
4  2.746755e+07  [-28.0, -28.0, -26.0, -28.0, -26.0, -26.0, -30...   

                                            distance  \
0  [  2.6799998   3.12        3.1599998   3.21   ...   
1  [  2.6299999   2.6799998   2.7         3.01   ...   
2  [  3.22        3.28        3.29        3.3    ...   
3  [  3.12        3.12        3.25        3.26   ...   
4  [  3.26        3.28        3.31        3.39999...   

                                        angleAzimuth  \
0  [-0.9527408  -0.5877923  -0.8913226  -0.933158...   
1  [-1.0275631e+00 -1.0343698e+00 -9.2038232e-01 ...   
2  [-4.

In [9]:
# Define relevant columns
relevant_columns = ["x_det", "y_det", "radialVelocity", "rcs", 
                    "ego_speed", 
                    "orientation", "width_edge_mean", "length_edge_mean", "centroid_x", "centroid_y"]

# Keep only relevant columns
df_cleaned = df[relevant_columns]

# Define the new file path
new_csv_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan.csv"

# Save cleaned data
df_cleaned.to_csv(new_csv_path, index=False)

print(f"Cleaned dataset saved as: {new_csv_path}")
print("\nSample Data:\n", df_cleaned.head())

Cleaned dataset saved as: /home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan.csv

Sample Data:
                                                x_det  \
0  [5.0138464, 6.054301, 5.4824324, 5.402544, 6.6...   
1  [4.849637, 4.8607, 5.1273346, 5.1149383, 5.930...   
2  [6.6600795, 6.017455, 5.9280252, 5.953123, 6.7...   
3  [5.7291083, 5.6479535, 5.2827706, 6.0945125, 6...   
4  [5.894352, 5.8527813, 6.0481644, 6.1411476, 6....   

                                               y_det  \
0  [-2.4095554, -1.9927621, -2.7368078, -2.849057...   
1  [-2.51193047, -2.5649507, -2.42041469, -2.6207...   
2  [-0.436480641, -2.35650873, -2.48714042, -2.47...   
3  [-2.44097495, -2.52830172, -3.00039482, -2.190...   
4  [-2.47028732, -2.55081868, -2.33068514, -2.405...   

                                      radialVelocity  \
0  [-2.1499999, -2.07, -1.5949999, -1.475, -2.215...   
1  [-2.48, -2.4649999, -3.04, -2.8299999, -3.8099...   
2  [-5.

In [11]:


# Load the cleaned dataset
csv_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan.csv"
df_cleaned = pd.read_csv(csv_path)

# Split into 70% Train, 15% Validation, 15% Test
train_df, temp_df = train_test_split(df_cleaned, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save splits as new CSV files
train_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv"
val_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_val.csv"
test_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_test.csv"

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

# Print dataset sizes
print(f"Training Set: {len(train_df)} rows saved to {train_path}")
print(f"Validation Set: {len(val_df)} rows saved to {val_path}")
print(f"Test Set: {len(test_df)} rows saved to {test_path}")

Training Set: 27155 rows saved to /home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv
Validation Set: 5819 rows saved to /home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_val.csv
Test Set: 5820 rows saved to /home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_test.csv


In [18]:


# Load datasets
train_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv"
val_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_val.csv"
test_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_test.csv"

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

# Convert detection features (list-based columns)
def convert_to_list(value):
    """Ensure values are lists. If string, clean and convert to list of floats."""
    if isinstance(value, list):  
        return value  # Already a list
    elif isinstance(value, str):  
        value = value.replace("[", "").replace("]", "").replace(",", " ")  # Clean brackets and commas
        return [float(x) for x in value.split() if x.strip()]  # Convert to floats
    else:
        return []  # Return empty list if unexpected format

# Convert label columns (single float per row)
def extract_single_value(value):
    """Extract a single float from a list or string representation."""
    if isinstance(value, list):  
        return float(value[0])  # Extract first value if already a list
    elif isinstance(value, str):  
        return float(value.replace("[", "").replace("]", ""))  # Remove brackets and convert
    else:
        return float(value)  # Convert directly if already a float

# Apply to all datasets (train, validation, test)
for df in [train_df, val_df, test_df]:
    # Apply conversion to detection columns (input features)
    for col in ["rcs", "x_det", "y_det", "radialVelocity"]:
        df[col] = df[col].apply(convert_to_list)

    # Apply conversion to label columns (target outputs)
    for col in ["orientation", "width_edge_mean", "length_edge_mean", "centroid_x", "centroid_y"]:
        df[col] = df[col].apply(extract_single_value)

    # Ensure scalar feature remains float (ego_speed)
    df["ego_speed"] = df["ego_speed"].astype(float)

# Save cleaned datasets
train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("✅ Input features and labels successfully converted and saved for train, validation, and test sets!")

✅ Input features and labels successfully converted and saved for train, validation, and test sets!


In [19]:

# Load datasets
train_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv"
val_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_val.csv"
test_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_test.csv"

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

# Convert list-based detection columns back to lists
def convert_to_list(value):
    """Ensure values are lists. If stored as string, convert back to list of floats."""
    if isinstance(value, list):
        return value
    elif isinstance(value, str):
        value = value.replace("[", "").replace("]", "").replace(",", " ")
        return [float(x) for x in value.split() if x.strip()]
    else:
        return []

# Apply conversion to detection columns
for df in [train_df, val_df, test_df]:
    for col in ["rcs", "x_det", "y_det", "radialVelocity"]:
        df[col] = df[col].apply(convert_to_list)

# Fit scaler on training set only (avoiding data leakage)
detection_scaler = StandardScaler()

# Flatten all training detections to fit the scaler
train_detections = np.vstack([
    np.column_stack([
        np.array(x), np.array(y), np.array(v), np.array(rcs)
    ])
    for x, y, v, rcs in zip(train_df["x_det"], train_df["y_det"], train_df["radialVelocity"], train_df["rcs"])
])

# Fit scaler on training detections
detection_scaler.fit(train_detections)

# Normalize all datasets
def normalize_detections(row):
    """Normalize detection features while maintaining variable-length structure."""
    detections = np.column_stack([
        row["x_det"], row["y_det"], row["radialVelocity"], row["rcs"]
    ])
    return detection_scaler.transform(detections).tolist()  # Keep as list for later use

for df in [train_df, val_df, test_df]:
    df["x_det"] = df.apply(normalize_detections, axis=1)

# Normalize ego_speed (fit on train, apply to all)
scalar_scaler = StandardScaler()
scalar_scaler.fit(train_df[["ego_speed"]])

train_df["ego_speed"] = scalar_scaler.transform(train_df[["ego_speed"]])
val_df["ego_speed"] = scalar_scaler.transform(val_df[["ego_speed"]])
test_df["ego_speed"] = scalar_scaler.transform(test_df[["ego_speed"]])

# Save the normalized datasets
train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("✅ Feature normalization completed and saved!")

✅ Feature normalization completed and saved!


In [21]:


# Load the cleaned and normalized datasets
train_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv"
val_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_val.csv"
test_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_test.csv"

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

# Convert detection columns from string to lists
def convert_to_list(value):
    """Convert stored string lists back to actual lists."""
    if isinstance(value, list):
        return value
    elif isinstance(value, str):
        value = value.replace("[", "").replace("]", "").replace(",", " ")
        return [float(x) for x in value.split() if x.strip()]
    else:
        return []

# Apply conversion to detection columns
for df in [train_df, val_df, test_df]:
    for col in ["rcs", "x_det", "y_det", "radialVelocity"]:
        df[col] = df[col].apply(convert_to_list)

# Convert dataset into TensorFlow-compatible format
def preprocess(row):
    """Process a single row into model input format."""
    # Stack detections into a (N_detections, 4) array
    detections = np.column_stack([
        row["x_det"],
        row["y_det"],
        row["radialVelocity"],
        row["rcs"]
    ])
    
    # Convert to TensorFlow format (RaggedTensor allows variable-length detections)
    detections_tensor = tf.ragged.constant(detections)

    # Scalar input: ego_speed
    ego_speed_tensor = tf.convert_to_tensor([row["ego_speed"]], dtype=tf.float32)

    # Labels
    labels = [
        row["centroid_x"],
        row["centroid_y"],
        row["orientation"],
        row["width_edge_mean"],
        row["length_edge_mean"]
    ]
    labels_tensor = tf.convert_to_tensor(labels, dtype=tf.float32)

    return {"detections": detections_tensor, "ego_speed": ego_speed_tensor}, labels_tensor

# Convert DataFrame to TensorFlow Dataset
def create_dataset(df):
    """Convert a Pandas DataFrame into a TensorFlow Dataset."""
    dataset = tf.data.Dataset.from_generator(
        lambda: (preprocess(row) for _, row in df.iterrows()),
        output_signature=(
            {
                "detections": tf.RaggedTensorSpec(shape=[None, 4], dtype=tf.float32),
                "ego_speed": tf.TensorSpec(shape=[1], dtype=tf.float32)
            },
            tf.TensorSpec(shape=[5], dtype=tf.float32)  # Labels
        )
    )
    return dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Create train, validation, and test datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)
test_dataset = create_dataset(test_df)

print("✅ TensorFlow datasets created successfully!")

2025-03-02 14:46:12.086700: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


✅ TensorFlow datasets created successfully!


In [26]:


class RadarPointNetPlusPlus(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        # Shared MLP for per-point feature extraction
        self.shared_mlp = tf.keras.Sequential([
            Dense(64, activation="relu"),
            Dense(128, activation="relu"),
            Dense(256, activation="relu")
        ])
        
        # Feature Aggregation (Global Pooling)
        self.global_pool = Lambda(lambda x: tf.reduce_max(x, axis=1))

        # Fully Connected Layers for Final Prediction
        self.fc = tf.keras.Sequential([
            Dense(256, activation="relu"),
            Dense(128, activation="relu"),
            Dense(5)  # Output: [centroid_x, centroid_y, orientation, width, length]
        ])
        
    def call(self, inputs):
        """Forward pass through the model."""
        detections = inputs["detections"]  # Expected: (batch_size, N_detections, 4)
        ego_speed = inputs["ego_speed"]    # Expected: (batch_size, 1)

        # Check if detections are RaggedTensor and convert only if necessary
        if isinstance(detections, tf.RaggedTensor):
            detections = detections.to_tensor()  # Convert to dense tensor only if ragged

        # Apply Shared MLP to each detection
        point_features = self.shared_mlp(detections)  # Shape: (batch_size, N_detections, 256)

        # Feature Aggregation (Max Pooling over detections)
        global_features = self.global_pool(point_features)  # Shape: (batch_size, 256)

        # Concatenate with ego_speed
        combined_features = Concatenate()([global_features, ego_speed])  # Shape: (batch_size, 257)

        # Fully Connected Layers for Final Prediction
        outputs = self.fc(combined_features)  # Shape: (batch_size, 5)

        return outputs

# Create model instance
model = RadarPointNetPlusPlus()

# Compile the model
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mae"]
)

# Fix: Corrected input shape format
model.build(input_shape={"detections": (None, None, 4), "ego_speed": (None, 1)})
model.summary()

Model: "radar_point_net_plus_plus_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, None, 256)         41664     
                                                                 
 lambda_2 (Lambda)           multiple                  0         
                                                                 
 sequential_5 (Sequential)   (None, 5)                 99589     
                                                                 
Total params: 141253 (551.77 KB)
Trainable params: 141253 (551.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# new pipeline with removed data bug

In [29]:
import numpy as np
import tensorflow as tf

def preprocess(row):
    """Process a single row, ensuring all detection features have the same length."""
    
    # Find the minimum length among all detection features in this row
    min_len = min(len(row["x_det"]), len(row["y_det"]), len(row["radialVelocity"]), len(row["rcs"]))

    # Truncate all lists to ensure they have the same length
    x_det = row["x_det"][:min_len]
    y_det = row["y_det"][:min_len]
    radialVelocity = row["radialVelocity"][:min_len]
    rcs = row["rcs"][:min_len]

    # Stack detections into a (N_detections, 4) array
    detections = np.column_stack([x_det, y_det, radialVelocity, rcs])

    # Convert to TensorFlow RaggedTensor
    detections_tensor = tf.ragged.constant(detections, dtype=tf.float32)

    # Scalar input: ego_speed
    ego_speed_tensor = tf.convert_to_tensor([row["ego_speed"]], dtype=tf.float32)

    # Labels (Ground truth outputs)
    labels = [
        row["centroid_x"],
        row["centroid_y"],
        row["orientation"],
        row["width_edge_mean"],
        row["length_edge_mean"]
    ]
    labels_tensor = tf.convert_to_tensor(labels, dtype=tf.float32)

    return {"detections": detections_tensor, "ego_speed": ego_speed_tensor}, labels_tensor

In [30]:
def create_dataset(df, batch_size=32):
    """Convert a Pandas DataFrame into a TensorFlow Dataset with batching."""
    dataset = tf.data.Dataset.from_generator(
        lambda: (preprocess(row) for _, row in df.iterrows()),
        output_signature=(
            {
                "detections": tf.RaggedTensorSpec(shape=[None, 4], dtype=tf.float32),
                "ego_speed": tf.TensorSpec(shape=[1], dtype=tf.float32)
            },
            tf.TensorSpec(shape=[5], dtype=tf.float32)  # Labels: (centroid_x, centroid_y, orientation, width, length)
        )
    )
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Recreate datasets using the fixed preprocessing function
train_dataset = create_dataset(train_df, batch_size=32)
val_dataset = create_dataset(val_df, batch_size=32)
test_dataset = create_dataset(test_df, batch_size=32)

print("✅ TensorFlow datasets created successfully with batch size 32!")

✅ TensorFlow datasets created successfully with batch size 32!


In [31]:
for sample in train_dataset.take(1):
    print("Batch Shape (detections):", sample[0]["detections"].shape)
    print("Batch Shape (ego_speed):", sample[0]["ego_speed"].shape)
    print("Batch Shape (labels):", sample[1].shape)

Batch Shape (detections): (32, None, 4)
Batch Shape (ego_speed): (32, 1)
Batch Shape (labels): (32, 5)


In [35]:



# Optional: Clear TensorFlow session to avoid memory issues
K.clear_session()
print("✅ TensorFlow session cleared!")

# Reinitialize the model to reset weights
model = RadarPointNetPlusPlus()

# Compile the model again
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mae"]
)

print("✅ Model weights have been reset!")

# Define batch size
batch_size = 32

# Calculate number of batches per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,  # You can adjust based on performance
    steps_per_epoch=steps_per_epoch,        # Fix for x/Unknown issue
    validation_steps=validation_steps,      # Fix for x/Unknown issue
    callbacks=[early_stopping]
)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
plt.show()

# Plot training and validation MAE
plt.figure(figsize=(10, 5))
plt.plot(history.history["mae"], label="Train MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.xlabel("Epochs")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Training and Validation MAE")
plt.legend()
plt.grid()
plt.show()

✅ TensorFlow session cleared!
✅ Model weights have been reset!
Epoch 1/50
848/848 [==============================] - 627s 737ms/step - loss: 599.7581 - mae: 10.3508 - val_loss: 282.0904 - val_mae: 7.2375
Epoch 2/50
  1/848 [..............................] - ETA: 38s - loss: 331.5275 - mae: 7.6212WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 42400 batches). You may need to use the repeat() function when building your dataset.


KeyboardInterrupt: 

In [36]:
import pandas as pd

# Load the CSV file (train dataset as example)
csv_path = "/home/q674749/workspace/thesis_work/rat25-15.4.1/perception/BERT_NOTEBOOK/final_leading_object_data/mohan_train.csv"
df = pd.read_csv(csv_path)

# Convert list-based features from string format to lists
def convert_to_list(value):
    """Ensure values are lists. If string, convert to list of floats."""
    if isinstance(value, list):
        return value
    elif isinstance(value, str):
        value = value.replace("[", "").replace("]", "").replace(",", " ")
        return [float(x) for x in value.split() if x.strip()]
    else:
        return []

# Apply conversion to detection columns
for col in ["x_det", "y_det", "radialVelocity", "rcs"]:
    df[col] = df[col].apply(convert_to_list)

# Check length consistency across rows
df["x_det_len"] = df["x_det"].apply(len)
df["y_det_len"] = df["y_det"].apply(len)
df["radialVelocity_len"] = df["radialVelocity"].apply(len)
df["rcs_len"] = df["rcs"].apply(len)

# Find rows where lengths are inconsistent
inconsistent_rows = df[
    (df["x_det_len"] != df["y_det_len"]) |
    (df["x_det_len"] != df["radialVelocity_len"]) |
    (df["x_det_len"] != df["rcs_len"])
]

# Print the number of inconsistent rows
print("Total Rows in Dataset:", len(df))
print("❗ Rows with inconsistent feature lengths:", len(inconsistent_rows))

Total Rows in Dataset: 27155
❗ Rows with inconsistent feature lengths: 27155
